## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,rikitea,PF,-23.1203,-134.9692,77.79,73,78,13.11,light rain,2022-01-06 23:43:01
1,1,khatanga,RU,71.9667,102.5000,-23.73,100,100,5.57,overcast clouds,2022-01-06 23:43:01
2,2,new norfolk,AU,-42.7826,147.0587,72.66,89,100,2.35,overcast clouds,2022-01-06 23:43:02
3,3,busselton,AU,-33.6500,115.3333,70.05,57,0,16.11,clear sky,2022-01-06 23:43:02
4,4,avarua,CK,-21.2078,-159.7750,82.45,78,100,6.91,overcast clouds,2022-01-06 23:43:03


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,rikitea,PF,-23.1203,-134.9692,77.79,73,78,13.11,light rain,2022-01-06 23:43:01
4,4,avarua,CK,-21.2078,-159.7750,82.45,78,100,6.91,overcast clouds,2022-01-06 23:43:03
5,5,hithadhoo,MV,-0.6000,73.0833,80.62,78,100,6.42,overcast clouds,2022-01-06 23:43:03
11,11,cockburn town,TC,21.4612,-71.1419,78.98,74,2,17.94,clear sky,2022-01-06 23:38:24
13,13,yulara,AU,-25.2406,130.9889,78.64,21,0,18.41,clear sky,2022-01-06 23:43:05
19,19,enugu,NG,6.4402,7.4943,75.06,59,6,5.35,clear sky,2022-01-06 23:43:07
20,20,iracoubo,GF,5.4802,-53.2011,76.44,89,93,13.00,overcast clouds,2022-01-06 23:43:08
23,23,carnarvon,AU,-24.8667,113.6333,78.87,73,100,17.27,overcast clouds,2022-01-06 23:43:09
27,27,touros,BR,-5.1989,-35.4608,81.50,76,63,10.51,broken clouds,2022-01-06 23:43:10
28,28,batemans bay,AU,-35.7167,150.1833,81.19,84,100,9.71,light rain,2022-01-06 23:43:10


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                183
City                   183
Country                183
Lat                    183
Lng                    183
Max Temp               183
Humidity               183
Cloudiness             183
Wind Speed             183
Current Description    183
Date                   183
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,rikitea,PF,77.79,light rain,-23.1203,-134.9692,
4,avarua,CK,82.45,overcast clouds,-21.2078,-159.7750,
5,hithadhoo,MV,80.62,overcast clouds,-0.6000,73.0833,
11,cockburn town,TC,78.98,clear sky,21.4612,-71.1419,
13,yulara,AU,78.64,clear sky,-25.2406,130.9889,
19,enugu,NG,75.06,clear sky,6.4402,7.4943,
20,iracoubo,GF,76.44,overcast clouds,5.4802,-53.2011,
23,carnarvon,AU,78.87,overcast clouds,-24.8667,113.6333,
27,touros,BR,81.50,broken clouds,-5.1989,-35.4608,
28,batemans bay,AU,81.19,light rain,-35.7167,150.1833,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [24]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,rikitea,PF,77.79,light rain,-23.1203,-134.9692,People ThankYou
4,avarua,CK,82.45,overcast clouds,-21.2078,-159.7750,Paradise Inn
5,hithadhoo,MV,80.62,overcast clouds,-0.6000,73.0833,Scoop Guest House
11,cockburn town,TC,78.98,clear sky,21.4612,-71.1419,Osprey Beach Hotel
13,yulara,AU,78.64,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
19,enugu,NG,75.06,clear sky,6.4402,7.4943,Bestchoice Hotel & Suites Enugu
20,iracoubo,GF,76.44,overcast clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
23,carnarvon,AU,78.87,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon
27,touros,BR,81.50,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
28,batemans bay,AU,81.19,light rain,-35.7167,150.1833,Batemans Bay Marina Resort


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))